In [93]:
import requests
from lxml import etree

article_list = []
for page in range(1,6):
    print(f'第{page}页')
    url = f'https://www.medindia.net/medicalwebsite/indian_healthcategory.asp?alpha=&page={page}&categoryid=9&subcategoryid=42'
    res = requests.get(url)
    html = etree.HTML(res.text)
    
    articles = html.xpath('//div[@class="report-content"]/article')
    for article in articles:
        name = article.xpath('./div/h4/text()')[0].strip().replace('\r', '')
        url  = article.xpath('./div/span/text()')[0].strip()
        print(url)
        if 'http' not in url:
            continue
        if name == '':
            name = 'Null'
        article_list.append({'NAME':name, 'URL':url})
article_list

第1页
200
http://www.tomarorthopaedics.com
http://amarjyothiclinic.org/index.html
http://dentalclinicdelhi.com

http://www.vitalradiology.com
http://www.pune-dentist.com
http://www.rescuehealthcare.in/
http://www.sankaraeye.com/
http://www.ratandeep.com
http://boneandjointcare.co.in/
第2页
200
http://www.andrology.com/sponsor.htm
http://www.apollohospitals.com
http://www.vellorecmc.org
http://www.choithram.org
http://www.sitalakshmi.com/hospital
http://www.dragarwal.com
http://www.jillahospital.com
http://www.kanti.com
http://www.kghospital.org
http://www.kotharimedical.com
第3页
200
http://members.tripod.com/
http://www.miothospitals.com/
http://www.mvdsc.org
http://www.dynacs.com/
http://www.sgpgi.ac.in
http://www.sksh.org
http://www.speechfoundation.com
http://www.smfhospital.org
http://www.tatamemorialcentre.com
http://www.vardaan.net
第4页
200
http://www.sahajdental.com


IndexError: list index out of range

# 八万所诊所测试

## 找出所有州

<img alt="图 1" src="../../images/fb079143228ea80523eee9dabf4c478e771c24373d6181f7275559671f85898d.png" />  


In [15]:
import requests
from lxml import etree

url = 'https://www.medindia.net/patients/hospital_search/hospital_list.asp?utm_source=topnavigation&utm_medium=desktop&utm_content=&utm_campaign=medindia'
res = requests.get(url)
print(res.status_code)
print(res.text)
html = etree.HTML(res.text)
lis = html.xpath('//ul[@class="list-inline"]/div/div/li/a')
for li in lis:
    name = li.xpath('./span/text()')[0]
    href = li.xpath('./@href')[0]
    print((name, href))

('andaman and nicobar islands', 'https://www.medindia.net/patients/hospital_search/hospitals-andaman-and-nicobar-islands.htm')
('andhra pradesh', 'https://www.medindia.net/patients/hospital_search/hospitals-andhra-pradesh.htm')
('arunachal pradesh', 'https://www.medindia.net/patients/hospital_search/hospitals-arunachal-pradesh.htm')
('assam', 'https://www.medindia.net/patients/hospital_search/hospitals-assam.htm')
('bihar', 'https://www.medindia.net/patients/hospital_search/hospitals-bihar.htm')
('chandigarh', 'https://www.medindia.net/patients/hospital_search/hospitals-chandigarh.htm')
('chhattisgarh', 'https://www.medindia.net/patients/hospital_search/hospitals-chhattisgarh.htm')
('dadra and nagar haveli', 'https://www.medindia.net/patients/hospital_search/hospitals-dadra-and-nagar-haveli.htm')
('daman and diu', 'https://www.medindia.net/patients/hospital_search/hospitals-daman-and-diu.htm')
('delhi', 'https://www.medindia.net/patients/hospital_search/hospitals-delhi.htm')
('goa', 'h

## 按地区划分（找出所有医院）

In [42]:
url = 'https://www.medindia.net/patients/hospital_search/hospitals-delhi.htm'

res = requests.get(url)
print(res.status_code)
html = etree.HTML(res.text)
h3s = html.xpath('//h3[@class="vert-small-margin"]')
for h3 in h3s:
    print('Name:', h3.xpath('./a/text()')[0])
    print('Href:', h3.xpath('./a/@href')[0])
    print('-'*100)

200
Name: al-hayat multispeciality hospital & gynae care, 
Href: https://www.medindia.net/patients/hospital_search/al-hayat-multispeciality-hospital-gynae-care-delhi-delhi-84868-1.htm
----------------------------------------------------------------------------------------------------
Name: american laser & skin clinic, 
Href: https://www.medindia.net/patients/hospital_search/american-laser-skin-clinic-delhi-delhi-86079-1.htm
----------------------------------------------------------------------------------------------------
Name: bhagwati homeopathic clinic, 
Href: https://www.medindia.net/patients/hospital_search/bhagwati-homeopathic-clinic-delhi-delhi-88994-1.htm
----------------------------------------------------------------------------------------------------
Name: brij hormone lab and thyroid center, 
Href: https://www.medindia.net/patients/hospital_search/brij-hormone-lab-and-thyroid-center-delhi-delhi-86633-1.htm
-----------------------------------------------------------------

## 具体的某一个医院

<img alt="图 4" src="../../images/3cb6a4f67cf67e98ef2ad4eb9971cccdfd58f5059867cabe4c8830fd033ad887.png" />  



In [81]:
url = 'https://www.medindia.net/patients/hospital_search/s-r-hospital-kullu-himachal-pradesh-27430-1.htm'

res = requests.get(url)
print(res.status_code)
html = etree.HTML(res.text)

name              = html.xpath('//div[@class="report-content"]/b/text()')[0]
addressinfo       = html.xpath('//div[contains(@class, "xs-horiz-medium-padding")]/div[1]/div[contains(@class, "report-content")]')[0]
# 拼接地址
address_join      = ''.join(addressinfo.xpath('./p[1]/text()'))
# 详细地址（具体地址、市-邮编、省（邦）
detail_address    = addressinfo.xpath('./p[1]/text()')[0]    # 包含地址、邮编、市
town_and_postcode = addressinfo.xpath('./p[1]/text()')[1]
province          = addressinfo.xpath('./p[1]/text()')[2]
try:
    phone         = addressinfo.xpath('./p[2]/span/text()')[0]
except:
    phone         = None
try:
    mobile        = addressinfo.xpath('./p[3]/span/text()')[0]
except:
    mobile        = None

print((name, address_join, phone, mobile))

200
('S R Hospital & Surgical Center', ' Nh-21, Vill. Kalheli- P. O Bajaura- Tehshil, Kullu - 175125, Himachal Pradesh', '1903-266564', None)


# 随机UA

In [7]:
# 构造随机函数
def get_ua():
    import random
    first_num = random.randint(55, 76)
    third_num = random.randint(0, 3800)
    fourth_num = random.randint(0, 140)
    os_type = ['(Windows NT 6.1; WOW64)', '(Windows NT 10.0; WOW64)', '(X11; Linux x86_64)', '(Macintosh; Intel Mac OS X 10_14_5)']
    chrome_version = 'Chrome/{}.0.{}.{}'.format(first_num, third_num, fourth_num)
    ua = ' '.join(['Mozilla/5.0', random.choice(os_type), 'AppleWebKit/537.36', '(KHTML, like Gecko)', chrome_version, 'Safari/537.36'])
    return ua

# 调用
get_ua()

'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3248.140 Safari/537.36'

In [2]:
import re

str = ' Cath LabGroundfloorCn Centre   , '
re.findall('\s*(\w.*\w)\s*,', str)

['Cath LabGroundfloorCn Centre']

In [33]:
import requests

html = requests.get('http://httpbin.org/get').text
print(html)

{
  "args": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.28.1", 
    "X-Amzn-Trace-Id": "Root=1-632199a9-7734a14242a59c366b396379"
  }, 
  "origin": "3.101.19.232", 
  "url": "http://httpbin.org/get"
}



# 代理

In [34]:
ips = [
    "218.102.244.20",
    "54.255.242.142",
    "13.231.127.120",
    "18.143.171.176",
    "16.162.122.38",
    "52.53.174.236",
    "35.78.169.15",
    "13.125.240.237",
    "18.162.152.81",
    "54.67.99.11",
    "35.78.201.245",
    "3.38.203.98",
    "13.212.83.105",
    "16.162.87.109",
    "3.101.19.232",
]

In [38]:
import requests

proxy = '127.0.0.1:7890'
proxies = {
    'http': 'http://' + proxy,
    'https': 'https://' + proxy,
}
try:
    response = requests.get('http://httpbin.org/get', proxies=proxies)
    print(response.text)
except requests.exceptions.ConnectionError as e:
    print('Error', e.args)

{
  "args": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.28.1", 
    "X-Amzn-Trace-Id": "Root=1-63219c18-37af74ab1a450d1059b0a085"
  }, 
  "origin": "13.212.83.105", 
  "url": "http://httpbin.org/get"
}



# 日志

In [1]:
import logging
logging.warning("This is a warning")

# 代理

In [4]:
import requests

# proxy = '127.0.0.1:7890'
# proxy = '47.106.105.236:80'
# 
proxies = {
    'http': 'http://' + proxy,
    'https': 'http://' + proxy,
}
try:
    response = requests.get('https://httpbin.org/get', proxies=proxies)
    print(response.text)
except requests.exceptions.ConnectionError as e:
    print('Error', e.args)

{
  "args": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.28.1", 
    "X-Amzn-Trace-Id": "Root=1-6322c03e-7706141122beef0e2e7eb931"
  }, 
  "origin": "61.150.96.27", 
  "url": "https://httpbin.org/get"
}



## 快代理

### 筛选可用代理

In [22]:
import requests
from lxml import etree
from time import sleep

great_proxy = []
for page in range(1, 6):
    print(f'正在抓取第{page}页')
    url = f'https://free.kuaidaili.com/free/inha/{page}/'
    try:
        html = etree.HTML(requests.get(url).text)
        table = html.xpath('//table[contains(@class, "table-bordered")]/tbody/tr')
        for td in table:
            ip   = td.xpath('./td[@data-title="IP"]/text()')[0]
            port = td.xpath('./td[@data-title="PORT"]/text()')[0]
            proxy = f'{ip}:{port}'

            proxies = {
                'http': 'http://' + proxy,
                'https': 'http://' + proxy,
            }
            try:
                response = requests.get('https://httpbin.org/get', proxies=proxies)
                great_proxy.append(proxy)
                print(proxy)
            except requests.exceptions.ConnectionError as e:
                print(f'【Error】{proxy}')
                continue  
    except:
        continue


正则抓取第1页
【Error】61.150.96.27:36880
【Error】61.216.185.88:60808
【Error】47.106.105.236:80
【Error】122.9.101.6:8888
【Error】223.82.60.202:8060
【Error】61.216.156.222:60808
【Error】47.105.91.226:8118
【Error】223.82.60.202:8060
【Error】60.170.204.30:8060
【Error】122.9.101.6:8888
【Error】47.106.105.236:80
【Error】47.106.105.236:80
【Error】47.106.105.236:80
【Error】47.106.105.236:80
【Error】150.109.32.166:80
正则抓取第2页
【Error】47.105.91.226:8118
【Error】223.82.60.202:8060
【Error】61.216.185.88:60808
120.194.55.139:6969
【Error】122.9.101.6:8888
101.200.127.149:3129
【Error】112.14.47.6:52024
101.200.127.149:3129
【Error】122.9.101.6:8888
【Error】122.9.101.6:8888
【Error】120.220.220.95:8085
【Error】47.105.91.226:8118
【Error】60.170.204.30:8060
【Error】223.82.60.202:8060
【Error】223.82.60.202:8060
正则抓取第3页
120.194.55.139:6969
【Error】47.106.105.236:80
【Error】120.194.55.139:6969
【Error】27.42.168.46:55481
【Error】223.96.90.216:8085
【Error】223.96.90.216:8085
【Error】210.5.10.87:53281
101.200.127.149:3129
【Error】47.106.105.236:80
【Er

# 崔庆才大佬的IP代理项目

In [3]:
import requests

proxypool_url = 'http://127.0.0.1:5555/random'
target_url = 'http://httpbin.org/ip'

def get_random_proxy():
    return requests.get(proxypool_url).text.strip()

def crawl(url, proxy):
    proxies = {'http': 'http://' + proxy}
    return requests.get(url, proxies=proxies).text


def main():
    proxy = get_random_proxy()
    # print('get random proxy', proxy)
    html = crawl(target_url, proxy)
    print(html)

main()

157.100.23.244:999


ConnectTimeout: HTTPConnectionPool(host='68.235.35.155', port=808): Max retries exceeded with url: http://httpbin.org/ip (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x00000175AF4617E0>, 'Connection to 68.235.35.155 timed out. (connect timeout=None)'))

In [7]:

ip = 'https://' + requests.get('http://127.0.0.1:5555/random').text.strip()
ip



'https://45.42.177.49:3128'